In [91]:
import polars as pl
import sys
import os
import time
import pathlib

In [6]:
sys.path.append('/Users/urolyi1/repos/march-madness')

In [7]:
import uro_cbb.bball_ref as bball_ref

In [8]:
%load_ext autoreload
%autoreload 2

In [71]:
postseason_df = bball_ref.download_basic_tournament_games(2024)
postseason_df

Team1,Team2,Score1,Score2,Box Score Link
str,str,i16,i16,str
"""UConn""","""Stetson""",91,52,"""https://www.sports-reference.c…"
"""Florida Atlantic""","""Northwestern""",65,77,"""https://www.sports-reference.c…"
"""San Diego State""","""UAB""",69,65,"""https://www.sports-reference.c…"
"""Auburn""","""Yale""",76,78,"""https://www.sports-reference.c…"
"""BYU""","""Duquesne""",67,71,"""https://www.sports-reference.c…"
…,…,…,…,…
"""Clemson""","""Arizona""",77,72,"""https://www.sports-reference.c…"
"""Alabama""","""Clemson""",89,82,"""https://www.sports-reference.c…"
"""UConn""","""Alabama""",86,72,"""https://www.sports-reference.c…"


In [69]:
def get_game_totals(row):
    link = row.select(pl.col('Box Score Link')).item()
    box_scores = bball_ref.download_box_score(link)
    basic1_total = box_scores.basic_box_score1[-1].with_columns(
        pl.lit(link).alias('link')
    )
    basic2_total = box_scores.basic_box_score2[-1].with_columns(
        pl.lit(link).alias('link')
    )
    return (
        row.select(
            pl.col('Team1').alias('Team'), pl.col('Score1').alias('PTS'), pl.col('Box Score Link').alias('link')
        )
        .vstack(
            row.select(
                pl.col('Team2').alias('Team'), pl.col('Score2').alias('PTS'), pl.col('Box Score Link').alias('link')
            )
        )
        .join(
            basic1_total.vstack(basic2_total),
            on=['PTS', 'link'],
            how='left'
        )
    )

In [79]:
game_totals = []
for i in range(len(postseason_df)):
    row = postseason_df[i]
    game_totals.append(get_game_totals(row))
    time.sleep(5)


In [127]:
pl.concat(game_totals)

Team,PTS,link,Player,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,GmSc
str,i16,str,str,i16,i16,i16,f32,i16,i16,f32,i16,i16,f32,i16,i16,f32,i16,i16,i16,i16,i16,i16,i16,i16,f32
"""UConn""",91,"""https://www.sports-reference.c…","""School Totals""",200,37,70,0.529,28,46,0.609,9,24,0.375,8,10,0.8,14,28,42,22,6,2,8,19,null
"""Stetson""",52,"""https://www.sports-reference.c…","""School Totals""",200,17,55,0.309,14,35,0.4,3,20,0.15,15,20,0.75,8,19,27,6,6,4,14,12,null
"""Florida Atlantic""",65,"""https://www.sports-reference.c…","""School Totals""",225,22,57,0.386,17,30,0.567,5,27,0.185,16,21,0.762,6,27,33,10,9,5,21,17,null
"""Northwestern""",77,"""https://www.sports-reference.c…","""School Totals""",225,29,69,0.42,21,46,0.457,8,23,0.348,11,15,0.733,13,26,39,20,12,1,12,13,null
"""San Diego State""",69,"""https://www.sports-reference.c…","""School Totals""",200,24,55,0.436,19,37,0.514,5,18,0.278,16,20,0.8,13,18,31,10,11,5,12,20,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Alabama""",72,"""https://www.sports-reference.c…","""School Totals""",200,26,58,0.448,15,35,0.429,11,23,0.478,9,11,0.818,7,21,28,9,2,5,7,15,null
"""Purdue""",63,"""https://www.sports-reference.c…","""School Totals""",200,22,55,0.4,12,30,0.4,10,25,0.4,9,10,0.9,10,28,38,13,5,2,14,8,null
"""NC State""",50,"""https://www.sports-reference.c…","""School Totals""",200,21,57,0.368,16,38,0.421,5,19,0.263,3,4,0.75,6,22,28,10,8,3,11,13,null


In [129]:
(
    pl.concat(game_totals)
    .select(
        pl.exclude(["link", "Player", "GmSc", "FG%", "2P%", "3P%", "FT%"]),
    )
    .with_columns(pl.lit(1).alias("G"))
    .group_by(["Team"])
    .agg(pl.all().sum())
    .sort('G')
)

Team,PTS,MP,FG,FGA,2P,2PA,3P,3PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,G
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32
"""Long Beach State""",65,200,24,72,21,55,3,17,14,22,13,31,44,11,6,3,12,14,1
"""St. Peter's""",49,200,16,55,12,31,4,24,13,19,7,11,18,3,4,1,6,22,1
"""Kentucky""",76,200,26,61,17,33,9,28,15,20,10,28,38,17,2,6,11,18,1
"""South Dakota State""",65,200,23,53,13,29,10,24,9,11,6,18,24,15,4,2,15,13,1
"""Grambling""",50,200,20,55,17,43,3,12,7,10,3,17,20,11,8,6,5,19,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Duke""",275,800,91,216,60,133,31,83,62,85,36,99,135,58,29,14,36,66,4
"""NC State""",352,1025,131,290,100,196,31,94,59,81,38,136,174,74,30,22,45,75,5
"""Alabama""",431,1000,147,312,88,173,59,139,78,112,54,123,177,72,24,29,51,98,5


In [89]:
advanced_df = bball_ref.download_archive_basketball_reference_advanced_stats_data(2024)
advanced_df

School,G,W,L,W-L%,SRS,SOS,ConfW,ConfL,HomeW,HomeL,AwayW,AwayL,Tm.,Opp.,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
str,i16,i16,i16,f32,f32,f32,i16,i16,i16,i16,i16,i16,i16,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""Abilene Christian""",32,15,17,0.469,-3.82,-1.18,10,10,8,6,5,9,2320,2351,71.300003,100.5,0.388,0.258,0.522,47.700001,49.900002,11.0,5.3,0.476,15.4,26.799999,0.284
"""Air Force""",31,9,22,0.29,-4.34,1.86,2,16,4,12,5,8,2051,2243,63.400002,103.599998,0.296,0.472,0.56,46.200001,62.0,10.2,11.8,0.538,16.9,23.799999,0.202
"""Akron""",34,24,10,0.706,3.01,-2.52,13,5,13,1,7,6,2517,2239,67.0,110.099998,0.326,0.428,0.558,53.299999,50.200001,8.4,8.1,0.526,14.7,30.9,0.237
"""Alabama""",32,21,11,0.656,21.1,11.42,13,5,14,2,5,5,2904,2594,74.199997,121.300003,0.352,0.467,0.6,53.0,52.099998,9.7,10.5,0.562,13.7,35.0,0.276
"""Alabama A&M""",33,11,22,0.333,-14.66,-7.57,9,9,7,5,3,14,2267,2501,71.800003,95.400002,0.479,0.27,0.511,49.599998,45.700001,10.6,10.7,0.455,19.4,31.700001,0.344
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Wright State""",32,18,14,0.563,0.0,-3.4,13,7,9,7,7,6,2767,2599,72.599998,117.699997,0.328,0.292,0.615,51.599998,51.200001,8.5,7.2,0.586,14.4,28.6,0.248
"""Wyoming""",32,15,17,0.469,0.96,4.99,8,10,9,5,3,10,2331,2383,67.900002,106.400002,0.391,0.369,0.56,50.799999,47.299999,7.5,7.0,0.519,16.4,29.1,0.289
"""Xavier""",33,16,17,0.485,12.24,10.94,9,11,11,7,3,8,2503,2460,71.400002,106.199997,0.32,0.32,0.528,51.5,59.900002,9.8,8.4,0.49,13.9,32.099998,0.238


In [88]:
stats_df = bball_ref.download_basketball_reference_stats_data(2024)
stats_df

School,G,W,L,W-L%,SRS,SOS,ConfW,ConfL,HomeW,HomeL,AwayW,AwayL,Tm.,Opp.,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
str,i16,i16,i16,f32,f32,f32,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,f32,i16,i16,f32,i16,i16,f32,i16,i16,i16,i16,i16,i16,i16
"""Abilene Christian""",34,16,18,0.471,-4.12,-1.12,10,10,8,6,5,10,2452,2500,1375,861,1987,0.433,180,530,0.34,550,757,0.727,329,1139,431,262,72,434,665
"""Air Force""",31,9,22,0.29,-4.77,1.43,2,16,4,12,5,8,2051,2243,1250,727,1604,0.453,273,757,0.361,324,475,0.682,225,872,451,202,122,372,541
"""Akron""",35,24,11,0.686,2.77,-2.08,13,5,13,1,7,6,2577,2316,1405,915,2018,0.453,280,863,0.324,467,642,0.727,363,1278,455,197,100,394,583
"""Alabama""",37,25,12,0.676,20.690001,11.8,13,5,14,2,5,5,3335,3006,1490,1136,2385,0.476,413,1108,0.373,650,842,0.772,472,1467,587,256,162,438,734
"""Alabama A&M""",35,12,23,0.343,-14.35,-7.52,9,9,7,5,4,15,2414,2653,1405,806,1918,0.42,148,516,0.287,654,906,0.722,394,1248,369,261,136,565,727
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Wright State""",32,18,14,0.563,-0.22,-3.62,13,7,9,7,7,6,2767,2599,1295,1033,1948,0.53,218,569,0.383,483,638,0.757,290,1111,529,200,90,379,499
"""Wyoming""",32,15,17,0.469,0.54,4.57,8,10,9,5,3,10,2331,2383,1290,791,1755,0.451,241,648,0.372,508,686,0.741,307,1103,374,165,80,408,500
"""Xavier""",34,16,18,0.471,12.23,11.03,9,11,11,7,3,9,2579,2538,1360,923,2116,0.436,232,678,0.342,501,672,0.746,401,1297,551,236,115,396,572


In [92]:
tournament_totals = pl.read_parquet(pathlib.Path('../data/bball_ref/raw/tournament_totals/tournament_team_total_stats_2024.parquet'))

In [93]:
tournament_totals

Team,PTS,MP,FG,FGA,2P,2PA,3P,3PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS_right,G
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32
"""Longwood""",92,400,45,94,31,58,14,36,28,38,14,28,42,25,16,6,26,29,132,2
"""St. Peter's""",98,400,43,108,28,60,15,48,31,44,16,41,57,22,8,6,21,39,132,2
"""Florida""",200,400,69,122,52,87,17,35,47,55,15,41,56,49,14,6,18,41,202,2
"""Western Kentucky""",138,400,61,138,40,78,21,60,13,21,22,55,77,40,14,15,28,24,156,2
"""Mississippi State""",102,400,47,108,31,58,16,50,10,16,21,37,58,27,16,5,27,26,120,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Illinois""",596,1600,213,486,154,310,59,176,91,131,68,185,253,122,49,28,65,124,576,8
"""Alabama""",862,2000,291,655,190,380,101,275,160,229,115,236,351,145,60,53,91,196,843,10
"""NC State""",704,2050,241,594,172,358,69,236,126,169,85,257,342,134,53,40,92,154,677,10


In [150]:
stats_df = bball_ref.download_basketball_reference_stats_data(2024)


AttributeError: 'NoneType' object has no attribute 'find_all'

In [181]:
conversion = {"UNC Greensboro": "North Carolina-Greensboro", "UNC":"North Carolina","Penn":"Pennsylvania",\
              "NC State": "North Carolina State","TCU":"Texas Christian","UMBC":"Maryland-Baltimore County",\
              "VCU":"Virginia Commonwealth","Saint Mary's":"Saint Mary's (CA)","UNC Wilmington":"North Carolina-Wilmington",\
              "ETSU":"East Tennessee State","USC":"Southern California","SMU":"Southern Methodist",\
              "St. Joseph's":"Saint Joseph's", "St. Peter's": "Saint Peter's","UConn":"Connecticut","California":"University of California",\
              "UNC Asheville":"North Carolina-Asheville","Pitt":"Pittsburgh","Ole Miss":"Mississippi",\
              "LSU":"Louisiana State",'BYU':"Brigham Young",'UMass':"Massachusetts","UNLV":"Nevada-Las Vegas",\
              "LIU-Brooklyn":"Long Island University","Detroit":"Detroit Mercy","Southern Miss":"Southern Mississippi",\
              "Texas A&M;":"Texas A&M","North Carolina A&T;":"North Carolina A&T"}

In [182]:
tournament_df = pl.read_parquet('../data/bball_ref/raw/tournament_totals/tournament_team_total_stats_2024.parquet')
tournament_df = tournament_df.with_columns(pl.col('Team').alias('School')).vstack(
    tournament_df.with_columns(
        pl.col('Team').map_elements(lambda x: conversion.get(x), return_dtype=str).alias('School')
    ).filter(~pl.col('School').is_null())
)
tournament_df

Team,PTS,MP,FG,FGA,2P,2PA,3P,3PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,G,School
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32,str
"""Longwood""",46,200,14,41,11,28,3,13,15,16,4,10,14,9,4,1,18,16,1,"""Longwood"""
"""Samford""",89,200,30,73,14,36,16,37,13,23,9,21,30,23,7,1,7,20,1,"""Samford"""
"""Texas Tech""",67,200,24,62,17,31,7,31,12,14,9,21,30,15,3,2,9,20,1,"""Texas Tech"""
"""Florida Atlantic""",65,225,22,57,17,30,5,27,16,21,6,27,33,10,9,5,21,17,1,"""Florida Atlantic"""
"""New Mexico""",56,200,19,64,16,41,3,23,15,23,15,24,39,6,4,2,13,16,1,"""New Mexico"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""St. Peter's""",49,200,16,55,12,31,4,24,13,19,7,11,18,3,4,1,6,22,1,"""Saint Peter's"""
"""TCU""",72,200,24,67,17,45,7,22,17,22,18,17,35,13,6,4,13,13,1,"""Texas Christian"""
"""UNC""",262,600,91,198,60,122,31,76,49,59,34,74,108,43,17,12,21,45,3,"""North Carolina"""


In [183]:
tournament_names = set(tournament_df.select(pl.col('School')).to_series().to_list())
stats_names = set(stats_df.select(pl.col('School')).to_series().to_list())
print(tournament_names - stats_names)

{'Texas Christian', "St. Peter's", 'North Carolina State', "Saint Mary's", 'UNC', 'UConn', 'BYU'}


In [184]:
stats_names

{'Abilene Christian',
 'Air Force',
 'Akron',
 'Alabama',
 'Alabama A&M',
 'Alabama State',
 'Albany (NY)',
 'Alcorn State',
 'American',
 'Appalachian State',
 'Arizona',
 'Arizona State',
 'Arkansas',
 'Arkansas State',
 'Arkansas-Pine Bluff',
 'Army',
 'Auburn',
 'Austin Peay',
 'Ball State',
 'Baylor',
 'Bellarmine',
 'Belmont',
 'Bethune-Cookman',
 'Binghamton',
 'Boise State',
 'Boston College',
 'Boston University',
 'Bowling Green State',
 'Bradley',
 'Brigham Young',
 'Brown',
 'Bryant',
 'Bucknell',
 'Buffalo',
 'Butler',
 'Cal Poly',
 'Cal State Bakersfield',
 'Cal State Fullerton',
 'Cal State Northridge',
 'California',
 'California Baptist',
 'Campbell',
 'Canisius',
 'Central Arkansas',
 'Central Connecticut State',
 'Central Michigan',
 'Charleston Southern',
 'Charlotte',
 'Chattanooga',
 'Chicago State',
 'Cincinnati',
 'Clemson',
 'Cleveland State',
 'Coastal Carolina',
 'Colgate',
 'College of Charleston',
 'Colorado',
 'Colorado State',
 'Columbia',
 'Connecticut',

In [185]:
(
    stats_df.join(
        tournament_df,
        on='School',
        how='inner'
    )
    .with_columns(
        (pl.col('G') - pl.col('G_right')).alias('G'),
        (pl.col('W') - pl.col('G_right') + 1).alias('W'),
        (pl.col('L') - 1).alias('L'),
    )
    .with_columns(
        (pl.col('W') / (pl.col('W') + pl.col('L'))).alias('W-L%'),
        (pl.col('Tm.') - pl.col('PTS')).alias('PTS'),
        
    )
    # .select(
    #     pl.exclude('^.*_right$')
    # )
)
        

School,G,W,L,W-L%,SRS,SOS,ConfW,ConfL,HomeW,HomeL,AwayW,AwayL,Tm.,Opp.,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,Team,PTS,MP_right,FG_right,FGA_right,2P,2PA,3P_right,3PA_right,FT_right,FTA_right,ORB_right,DRB,TRB_right,AST_right,STL_right,BLK_right,TOV_right,PF_right,G_right
str,i32,i32,i16,f64,f32,f32,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,f32,i16,i16,f32,i16,i16,f32,i16,i16,i16,i16,i16,i16,i16,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32
"""Akron""",34,24,10,0.705882,2.77,-2.08,13,5,13,1,7,6,2577,2316,1405,915,2018,0.453,280,863,0.324,467,642,0.727,363,1278,455,197,100,394,583,"""Akron""",2517,200,25,66,19,38,6,28,4,6,9,17,26,8,5,1,4,20,1
"""Alabama""",32,21,11,0.65625,20.690001,11.8,13,5,14,2,5,5,3335,3006,1490,1136,2385,0.476,413,1108,0.373,650,842,0.772,472,1467,587,256,162,438,734,"""Alabama""",2904,1000,147,312,88,173,59,139,78,112,54,123,177,72,24,29,51,98,5
"""Arizona""",33,25,8,0.757576,24.540001,9.45,15,5,15,1,7,4,3137,2594,1465,1124,2321,0.484,284,776,0.366,605,844,0.717,471,1533,665,300,133,430,590,"""Arizona""",2902,600,83,192,57,111,26,81,43,62,32,84,116,50,27,18,32,51,3
"""Auburn""",34,27,7,0.794118,22.459999,7.66,13,5,15,1,5,5,2909,2391,1400,1010,2121,0.476,280,795,0.352,609,812,0.75,393,1323,622,258,215,374,678,"""Auburn""",2833,200,27,53,20,33,7,20,15,22,9,25,34,11,6,3,14,25,1
"""Baylor""",33,23,10,0.69697,19.5,10.71,11,7,15,2,4,5,2814,2489,1430,967,2001,0.483,301,773,0.389,579,791,0.732,399,1229,514,236,110,421,577,"""Baylor""",2658,400,54,111,32,57,22,54,26,40,18,44,62,24,14,4,22,31,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Wagner""",32,17,15,0.53125,-10.64,-8.58,7,9,7,5,9,10,2103,2083,1335,758,1913,0.396,238,727,0.327,349,496,0.704,369,1183,421,195,76,326,507,"""Wagner""",2041,200,23,58,16,40,7,18,9,15,7,13,20,8,5,2,7,13,1
"""Washington State""",33,24,9,0.727273,14.02,7.19,14,6,15,2,6,4,2574,2335,1410,948,2046,0.463,223,658,0.339,455,646,0.704,391,1317,435,181,172,383,582,"""Washington State""",2452,400,43,106,31,69,12,37,24,31,20,49,69,21,8,12,23,29,2
"""Western Kentucky""",33,22,11,0.666667,1.19,-2.91,8,8,12,2,6,7,2728,2531,1361,996,2138,0.466,233,686,0.34,503,699,0.72,361,1358,454,260,110,470,625,"""Western Kentucky""",2659,200,28,69,20,45,8,24,5,6,12,30,42,17,5,4,18,15,1
